#  Домашнее задание 2. Основные операторы PostgreSQL

## Шаг 1. Создать таблицы с перечисленными ниже структурами, используя CSV-файлы.

### ⚠️ Поскольку исходные данные содержат ошибки (например, в `products` множество записей с product_id=0), то данные будут грузиться AS IS, без установки Foreign Key

In [242]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="hw2",
    user="postgres",
    password="hw2"
)

with conn.cursor() as cur:
    cur.execute("SELECT 1")
    conn.commit()

print("Подключение к базе данных установлено")


Подключение к базе данных установлено


In [243]:
clear_tables_query = """
DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS order_items CASCADE;
"""

create_tables_query = """
CREATE TABLE customer (
    customer_id INTEGER PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    gender VARCHAR(10),
    dob DATE,
    job_title VARCHAR(200),
    job_industry_category VARCHAR(100),
    wealth_segment VARCHAR(50),
    deceased_indicator VARCHAR(1),
    owns_car VARCHAR(10),
    address TEXT,
    postcode INTEGER,
    state VARCHAR(50),
    country VARCHAR(100),
    property_valuation INTEGER
);

CREATE TABLE product (
    product_id INTEGER NOT NULL,
    brand VARCHAR(100),
    product_line VARCHAR(100),
    product_class VARCHAR(50),
    product_size VARCHAR(50),
    list_price NUMERIC(10,2) NOT NULL,
    standard_cost NUMERIC(10,2)
    -- NOTE: product_id повторяется и не может быть PRIMARY KEY
    -- (product_id, list_price) - уникальная комбинация для JOIN
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    online_order BOOLEAN,
    order_status VARCHAR(50) NOT NULL
);

CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity NUMERIC(10,2) NOT NULL,
    item_list_price_at_sale NUMERIC(10,2) NOT NULL,
    item_standard_cost_at_sale NUMERIC(10,2)
);

-- Indexes для оптимизации запросов
CREATE INDEX idx_orders_customer_id ON orders(customer_id);
CREATE INDEX idx_orders_order_date ON orders(order_date);
CREATE INDEX idx_orders_online_order ON orders(online_order) WHERE online_order = TRUE;
CREATE INDEX idx_order_items_order_id ON order_items(order_id);
CREATE INDEX idx_order_items_product_id ON order_items(product_id);
CREATE INDEX idx_product_lookup ON product(product_id, list_price);  -- Для корректного JOIN
"""


with conn.cursor() as cur:
    cur.execute(clear_tables_query)
    cur.execute(create_tables_query)
    conn.commit()

with conn.cursor() as cur:
    # вывести все таблицы
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
    tables = cur.fetchall()
    for table in tables:
        print(table[0])

customer
product
orders
order_items


In [244]:
import pandas as pd

# Загрузка customer.csv
customer_df = pd.read_csv(
    "data/customer.csv",
    sep=";",
    na_values=["", "n/a", "N/A"],
    parse_dates=["DOB"]
)


In [245]:
from psycopg2.extras import execute_batch

# Подготовка данных для вставки
# Заменяем NaN на None для корректной обработки NULL в PostgreSQL
customer_data = customer_df.where(pd.notnull(customer_df), None)

# SQL запрос для вставки данных
insert_query = """
INSERT INTO customer (
    customer_id, first_name, last_name, gender, dob,
    job_title, job_industry_category, wealth_segment,
    deceased_indicator, owns_car, address, postcode,
    state, country, property_valuation
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)
"""

records = []
for _, row in customer_data.iterrows():    
    records.append((
        int(row['customer_id']),
        row['first_name'],
        row['last_name'],
        row['gender'],
        row['DOB'] if pd.notna(row['DOB']) else None,
        row['job_title'],
        row['job_industry_category'],
        row['wealth_segment'],
        row['deceased_indicator'],
        row['owns_car'],
        row['address'],
        int(row['postcode']) if pd.notna(row['postcode']) else None,
        row['state'],
        row['country'],
        int(row['property_valuation']) if pd.notna(row['property_valuation']) else None
    ))

# Загрузка данных в базу
with conn.cursor() as cur:
    execute_batch(cur, insert_query, records)
    conn.commit()


In [246]:
csv_rows = len(customer_df)

with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM customer")
    db_rows = cur.fetchone()[0]

print(f"Строк в CSV файле: {csv_rows:,}")
print(f"Строк в базе данных: {db_rows:,}")


Строк в CSV файле: 4,000
Строк в базе данных: 4,000


In [247]:
# Загрузка product.csv
product_df = pd.read_csv(
    "data/product.csv",
    na_values=["", "n/a", "N/A"]
)

In [248]:
# Подготовка данных для вставки
product_data = product_df.where(pd.notnull(product_df), None)

# SQL запрос для вставки данных
insert_query = """
INSERT INTO product (
    product_id, brand, product_line, product_class, product_size,
    list_price, standard_cost
) VALUES (
    %s, %s, %s, %s, %s, %s, %s
)
"""

records = []
for _, row in product_data.iterrows():
    records.append((
        int(row['product_id']),
        row['brand'],
        row['product_line'],
        row['product_class'],
        row['product_size'],
        float(row['list_price']) if pd.notna(row['list_price']) else None,
        float(row['standard_cost']) if pd.notna(row['standard_cost']) else None
    ))

# Загрузка данных в базу
with conn.cursor() as cur:
    execute_batch(cur, insert_query, records)
    conn.commit()


In [249]:
csv_rows = len(product_df)

with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM product")
    db_rows = cur.fetchone()[0]

print(f"Строк в CSV файле: {csv_rows:,}")
print(f"Строк в базе данных: {db_rows:,}")

Строк в CSV файле: 190
Строк в базе данных: 190


In [250]:
# Загрузка orders.csv
orders_df = pd.read_csv(
    "data/orders.csv",
    na_values=["", "n/a", "N/A"],
    parse_dates=["order_date"]
)

In [251]:
orders_data = orders_df.where(pd.notnull(orders_df), None)

orders_data['order_date'] = orders_data['order_date'].map(
    lambda x: x.date() if pd.notna(x) else None
)

insert_query = """
INSERT INTO orders (
    order_id, customer_id, order_date, online_order, order_status
) VALUES (
    %s, %s, %s, %s, %s
)
"""

records = []
for _, row in orders_data.iterrows():
    records.append((
        int(row['order_id']),
        int(row['customer_id']),
        row['order_date'],
        row['online_order'],
        row['order_status']
    ))

# Загрузка данных в базу
with conn.cursor() as cur:
    execute_batch(cur, insert_query, records, page_size=1000)
    conn.commit()


In [252]:
csv_rows = len(orders_df)

with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM orders")
    db_rows = cur.fetchone()[0]

print(f"Строк в CSV файле: {csv_rows:,}")
print(f"Строк в базе данных: {db_rows:,}")

Строк в CSV файле: 20,000
Строк в базе данных: 20,000


In [253]:
# Загрузка order_items.csv
order_items_df = pd.read_csv(
    "data/order_items.csv",
    na_values=["", "n/a", "N/A"]
)

In [254]:
order_items_data = order_items_df.where(pd.notnull(order_items_df), None)

insert_query = """
INSERT INTO order_items (
    order_item_id, order_id, product_id, quantity,
    item_list_price_at_sale, item_standard_cost_at_sale
) VALUES (
    %s, %s, %s, %s, %s, %s
)
"""

records = []
for _, row in order_items_data.iterrows():
    records.append((
        int(row['order_item_id']),
        int(row['order_id']),
        int(row['product_id']),
        float(row['quantity']) if pd.notna(row['quantity']) else None,
        float(row['item_list_price_at_sale']) if pd.notna(row['item_list_price_at_sale']) else None,
        float(row['item_standard_cost_at_sale']) if pd.notna(row['item_standard_cost_at_sale']) else None
    ))

# Загрузка данных в базу
with conn.cursor() as cur:
    execute_batch(cur, insert_query, records, page_size=1000)
    conn.commit()


In [255]:
csv_rows = len(order_items_df)

with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM order_items")
    db_rows = cur.fetchone()[0]

print(f"Строк в CSV файле: {csv_rows:,}")
print(f"Строк в базе данных: {db_rows:,}")

Строк в CSV файле: 20,000
Строк в базе данных: 20,000


In [256]:
from pprint import pprint

def run_query_and_print(query):
    with conn.cursor() as cur:
        cur.execute(query)
        result = cur.fetchall()
        pprint(result)


### Вывести все уникальные бренды, у которых есть хотя бы один продукт со стандартной стоимостью выше 1500 долларов, и суммарными продажами не менее 1000 единиц.


In [257]:
query = """
SELECT DISTINCT p.brand
FROM product p
JOIN order_items oi ON p.product_id = oi.product_id 
                    AND p.list_price = oi.item_list_price_at_sale
WHERE p.standard_cost > 1500
GROUP BY p.brand
HAVING SUM(oi.quantity) >= 1000;
"""

run_query_and_print(query)

[('Giant Bicycles',), ('OHM Cycles',), ('Solex',), ('Trek Bicycles',)]


### Для каждого дня в диапазоне с 2017-04-01 по 2017-04-09 включительно вывести количество подтвержденных онлайн-заказов и количество уникальных клиентов, совершивших эти заказы.

In [258]:
query = """
SELECT 
    o.order_date,
    COUNT(o.order_id) as online_orders,
    COUNT(o.customer_id) as unique_customers
from orders o
WHERE o.order_date BETWEEN '2017-04-01' AND '2017-04-09'
    AND o.online_order = TRUE
    AND o.order_status = 'Approved'
group by o.order_date
order by o.order_date;
"""
run_query_and_print(query)

[(datetime.date(2017, 4, 1), 37, 37),
 (datetime.date(2017, 4, 2), 29, 29),
 (datetime.date(2017, 4, 3), 27, 27),
 (datetime.date(2017, 4, 4), 32, 32),
 (datetime.date(2017, 4, 5), 33, 33),
 (datetime.date(2017, 4, 6), 36, 36),
 (datetime.date(2017, 4, 7), 24, 24),
 (datetime.date(2017, 4, 8), 33, 33),
 (datetime.date(2017, 4, 9), 30, 30)]


### Вывести профессии клиентов из сферы IT, чья профессия начинается с Senior; из сферы Financial Services, чья профессия начинается с Lead. Для обеих групп учитывать только клиентов старше 35 лет. Объединить выборки с помощью UNION ALL.

In [259]:
query = """
SELECT DISTINCT c.job_title
FROM customer c
WHERE c.job_industry_category = 'IT'
    AND c.job_title LIKE 'Senior%'
    AND DATE_PART('year', AGE(CURRENT_DATE, c.dob::DATE)) > 35

UNION ALL

SELECT DISTINCT c.job_title
FROM customer c
WHERE c.job_industry_category = 'Financial Services'
    AND c.job_title LIKE 'Lead%'
    AND DATE_PART('year', AGE(CURRENT_DATE, c.dob::DATE)) > 35;
"""

run_query_and_print(query)

[('Senior Developer',), ('Senior Sales Associate',)]


### Вывести бренды, которые были куплены клиентами из сферы Financial Services, но не были куплены клиентами из сферы IT.

In [261]:
query = """
SELECT DISTINCT p.brand
FROM product p
JOIN order_items oi ON p.product_id = oi.product_id 
                    AND p.list_price = oi.item_list_price_at_sale
JOIN orders o ON oi.order_id = o.order_id
JOIN customer c ON o.customer_id = c.customer_id
WHERE c.job_industry_category = 'Financial Services'

EXCEPT

SELECT DISTINCT p.brand
FROM product p
JOIN order_items oi ON p.product_id = oi.product_id 
                    AND p.list_price = oi.item_list_price_at_sale
JOIN orders o ON oi.order_id = o.order_id
JOIN customer c ON o.customer_id = c.customer_id
WHERE c.job_industry_category = 'IT';
"""
run_query_and_print(query)

[]


Показалось странным, что в ответе пустое множество, поэтому рассмотрим оба запроса отдельно:

In [262]:
query_it = """
SELECT DISTINCT p.brand
FROM product p
JOIN order_items oi ON p.product_id = oi.product_id 
                    AND p.list_price = oi.item_list_price_at_sale
JOIN orders o ON oi.order_id = o.order_id
JOIN customer c ON o.customer_id = c.customer_id
WHERE c.job_industry_category = 'Financial Services'
"""

run_query_and_print(query_it)

query_fs = """
SELECT DISTINCT p.brand
FROM product p
JOIN order_items oi ON p.product_id = oi.product_id 
                    AND p.list_price = oi.item_list_price_at_sale
JOIN orders o ON oi.order_id = o.order_id
JOIN customer c ON o.customer_id = c.customer_id
WHERE c.job_industry_category = 'IT';
"""

run_query_and_print(query_fs)

[('Giant Bicycles',),
 ('Norco Bicycles',),
 ('OHM Cycles',),
 ('Solex',),
 ('Trek Bicycles',),
 ('WeareA2B',)]
[('Giant Bicycles',),
 ('Norco Bicycles',),
 ('OHM Cycles',),
 ('Solex',),
 ('Trek Bicycles',),
 ('WeareA2B',)]


Видимо, у них совпадают вкусы.

### Вывести 10 клиентов (ID, имя, фамилия), которые совершили наибольшее количество онлайн-заказов (в штуках) брендов Giant Bicycles, Norco Bicycles, Trek Bicycles, при условии, что они активны и имеют оценку имущества (property_valuation) выше среднего среди клиентов из того же штата

In [263]:
query = """
WITH state_avg AS (
    SELECT 
        state,
        AVG(property_valuation) AS avg_valuation
    FROM customer
    GROUP BY state
)
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    COUNT(DISTINCT o.order_id) AS order_count
FROM customer c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN product p ON oi.product_id = p.product_id 
               AND oi.item_list_price_at_sale = p.list_price
JOIN state_avg sa ON c.state = sa.state
WHERE o.online_order = TRUE
    AND p.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
    AND c.deceased_indicator = 'N'
    AND c.property_valuation > sa.avg_valuation
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY order_count DESC
LIMIT 10;
"""

run_query_and_print(query)

[(2498, 'Rosana', 'Emmatt', 6),
 (2841, 'Bibby', 'Carnson', 6),
 (2877, 'Vaughan', 'Frank', 6),
 (2226, 'Hedvig', 'Flatt', 5),
 (1683, 'Brenn', 'Bacon', 5),
 (1988, 'Kelwin', 'Goldsmith', 5),
 (1570, 'Phil', 'Dodle', 5),
 (464, 'Karel', 'Dimelow', 5),
 (1517, 'Murdoch', 'Twort', 5),
 (446, 'Marilee', 'Oosthout de Vree', 5)]


### Вывести всех клиентов (ID, имя, фамилия), у которых нет подтвержденных онлайн-заказов за последний год, но при этом они владеют автомобилем и их сегмент благосостояния не Mass Customer.

In [265]:
query = """
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name
FROM customer c
WHERE c.owns_car = 'Yes'
    AND c.wealth_segment != 'Mass Customer'
    AND NOT EXISTS (
        SELECT 1
        FROM orders o
        WHERE o.customer_id = c.customer_id
            AND o.online_order = TRUE
            AND o.order_status = 'Approved'
            AND o.order_date >= (
                SELECT DATE(MAX(order_date) - INTERVAL '1 year')
                FROM orders
            )
    );
"""

run_query_and_print(query)

[(3565, 'Charyl', 'Pottiphar'),
 (3611, 'Uriah', 'Chantree'),
 (3795, 'Taylor', 'Dollimore'),
 (3652, 'Aldrich', 'Camble'),
 (3949, 'Costa', 'Sleightholm'),
 (3704, 'Haslett', 'Ropars'),
 (3322, 'Hew', 'Sworder'),
 (3989, 'Nicolas', 'Burdass'),
 (3715, 'Willow', 'Rusbridge'),
 (3895, 'Reginald', 'Struys'),
 (667, 'Nicoline', 'Samwayes'),
 (1487, 'Kaela', 'Soppett'),
 (3924, 'Perry', 'Lampert'),
 (3685, 'Solomon', 'Ruffles'),
 (3604, 'Findlay', 'Lubman'),
 (3229, 'Carrol', 'Gheorghie'),
 (3510, 'Jemima', 'Izaac'),
 (3533, 'Neale', 'Lowings'),
 (163, 'Kessia', 'Helder'),
 (3564, 'Sandy', 'Olford'),
 (71, 'Hoyt', 'Glavias'),
 (689, 'Hamnet', 'Berford'),
 (3598, 'Penny', 'McKirdy'),
 (2649, 'Merwyn', 'MacPhail'),
 (1917, 'Raquela', 'Schimpke'),
 (2789, 'Violetta', 'Bucksey'),
 (3838, 'Mil', 'Clemitt'),
 (3997, 'Blanch', 'Nisuis'),
 (3203, 'Swen', 'Merioth'),
 (3793, 'Shannen', 'Pitcock'),
 (3931, 'Kylie', 'Epine'),
 (2362, 'Herbie', 'Machans'),
 (65, 'Yale', 'Tanser'),
 (3521, 'Oneida', 'O

### Вывести всех клиентов из сферы 'IT' (ID, имя, фамилия), которые купили 2 из 5 продуктов с самой высокой list_price в продуктовой линейке Road.

In [275]:
query = """
WITH top5 AS (
    SELECT product_id, list_price
    FROM product
    WHERE product_line = 'Road'
    ORDER BY list_price DESC
    LIMIT 5
)
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name
FROM customer c
WHERE c.job_industry_category = 'IT'
    AND (
        SELECT COUNT(DISTINCT (p.product_id, p.list_price))
        FROM orders o
        JOIN order_items oi ON o.order_id = oi.order_id
        JOIN product p ON oi.product_id = p.product_id 
                       AND oi.item_list_price_at_sale = p.list_price
        WHERE o.customer_id = c.customer_id
            AND (p.product_id, p.list_price) IN (SELECT product_id, list_price FROM top5)
    ) >= {min_count};
"""

for i in range(1, 6):
    print(f"Минимальное количество купленных продуктов: {i}")
    run_query_and_print(query.format(min_count=i))

Минимальное количество купленных продуктов: 1
[(135, 'Rosalia', 'Sigart'),
 (255, 'Keeley', 'Kruger'),
 (429, 'Berny', 'Bold'),
 (441, 'Felicle', 'Beneze'),
 (563, 'Rube', 'Itzhayek'),
 (569, 'Jaime', 'Warkup'),
 (712, 'Norine', 'Antonik'),
 (968, 'Wayne', 'Woodfin'),
 (983, 'Shaylyn', 'Riggs'),
 (1004, 'Colas', 'Pumfrett'),
 (1339, 'Mariam', 'Guiel'),
 (1445, 'Bettine', 'Yoselevitch'),
 (1683, 'Brenn', 'Bacon'),
 (1736, 'Garrek', 'Rowbotham'),
 (1791, 'Ninon', 'Van Der Hoog'),
 (1820, 'Yard', 'Teeney'),
 (1969, 'Dimitry', 'Kunkler'),
 (2190, 'Maressa', 'Kondrachenko'),
 (2426, 'Yancy', 'Ovett'),
 (2457, 'Benito', 'Vearnals'),
 (2540, 'Donavon', None),
 (3012, 'Devland', 'Probart'),
 (3288, 'Fair', 'Dewen'),
 (3406, 'Lucy', 'Lackmann')]
Минимальное количество купленных продуктов: 2
[]
Минимальное количество купленных продуктов: 3
[]
Минимальное количество купленных продуктов: 4
[]
Минимальное количество купленных продуктов: 5
[]


Добавлены примеры для 1, 2, 3, 4, 5 продуктов, чтобы показать, что запрос работает корректно.

### Вывести клиентов (ID, имя, фамилия, сфера деятельности) из сфер IT или Health, которые совершили не менее 3 подтвержденных заказов в период 2017-01-01 по 2017-03-01, и при этом их общий доход от этих заказов превышает 10 000 долларов. Разделить вывод на две группы (IT и Health) с помощью UNION.


In [276]:
query = """
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.job_industry_category
FROM customer c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
WHERE c.job_industry_category = 'IT'
    AND o.order_status = 'Approved'
    AND o.order_date BETWEEN '2017-01-01' AND '2017-03-01'
GROUP BY c.customer_id, c.first_name, c.last_name, c.job_industry_category
HAVING COUNT(DISTINCT o.order_id) >= 3
    AND SUM(oi.quantity * oi.item_list_price_at_sale) > 10000

UNION

SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.job_industry_category
FROM customer c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
WHERE c.job_industry_category = 'Health'
    AND o.order_status = 'Approved'
    AND o.order_date BETWEEN '2017-01-01' AND '2017-03-01'
GROUP BY c.customer_id, c.first_name, c.last_name, c.job_industry_category
HAVING COUNT(DISTINCT o.order_id) >= 3
    AND SUM(oi.quantity * oi.item_list_price_at_sale) > 10000;
"""

run_query_and_print(query)

[(394, 'Roanne', 'Cowthard', 'Health'),
 (1583, 'Krysta', "O' Reagan", 'IT'),
 (2932, 'Bradly', 'Deboick', 'IT'),
 (2292, 'Christie', 'MacClure', 'Health'),
 (255, 'Keeley', 'Kruger', 'IT'),
 (2788, 'Melantha', 'Pickburn', 'Health'),
 (3169, 'Maxim', 'Chavrin', 'IT'),
 (1291, 'Warner', 'Zuker', 'Health'),
 (424, 'Dennie', 'Eunson', 'Health'),
 (2012, 'Maisey', 'Lavell', 'Health'),
 (1959, 'Arlen', 'Klossek', 'Health'),
 (723, 'Madelina', 'Marte', 'Health'),
 (799, 'Harland', 'Spilisy', 'IT'),
 (1154, 'Chase', 'Youd', 'Health'),
 (173, 'Ebba', 'Hanselmann', 'Health'),
 (2815, 'Emery', 'Carlett', 'Health'),
 (1913, 'Tamiko', 'Fergie', 'Health'),
 (2353, 'Lyn', 'Luquet', 'Health'),
 (1791, 'Ninon', 'Van Der Hoog', 'IT'),
 (167, 'Nathalie', 'Tideswell', 'Health'),
 (607, 'Adelaida', 'Redmond', 'Health'),
 (1389, 'Reina', 'Drever', 'Health'),
 (2992, 'Crystal', 'Assur', 'Health'),
 (2039, 'Laureen', 'Blower', 'IT'),
 (64, 'Gerek', 'Yve', 'IT'),
 (1298, 'Guglielmo', 'Yele', 'Health'),
 (1533